In [1]:
import numpy as np

In [16]:
import pywt
from math import cos, sin, pi, sqrt, log

In [12]:
def DCT_II_f(k, N):
    return lambda x: np.cos(pi * (x + 0.5) * k / N)

def w(k, N):
    c = sqrt(2) ** np.sign(k)
    return [c * DCT_II_f(k, N)(i / float(N)) for i in range(N)]

def DCT1_qt(rows, cols):
    """DCT transform matrix"""
    return np.array([w(k, rows) for k in range(cols)]).T

In [13]:
def wavelet_transform_qt(rows, cols, wavelet='db1'):
    """
    Generate a matrix using the specified wavelet from the PyWavelets library.
    :param rows: Number of rows in the output matrix.
    :param cols: Number of columns in the output matrix.
    :param wavelet: Wavelet type to use. Default is 'db1' (Daubechies wavelet).
    :return: Wavelet transform matrix.
    """
    wavelet = pywt.Wavelet(wavelet)
    wavelet_basis = pywt.wavedec(np.eye(rows, cols), wavelet, level=int(log(rows, 2)))

    # Flatten and format the wavelet basis into a matrix similar to Haar/DCT
    wavelet_matrix = np.array(wavelet_basis[0]) 
    return wavelet_matrix

In [24]:
def DCT1_wavelet_qt(rows, cols, wavelet='db1', shuffle=False):
    """Combined DCT and alternative wavelet transform matrix with optional column shuffling."""
    dct_matrix = DCT1_qt(rows, cols // 2)  
    print("dct_matrix.shape", dct_matrix.shape)
    wavelet_matrix = wavelet_transform_qt(rows, cols // 2, wavelet)
    print("wavelet_matrix.shape", wavelet_matrix.shape)

    # Hacemos un diccionario híbrido con dos transformadas pero si luego
    # en el OMP hacemos un cut en las columnas tomamos solamente la primera transformada
    # por eso quizás lo mejor sea hacer un shuffle antes

    combined_matrix = np.concatenate((dct_matrix, wavelet_matrix), axis=1)  

    if shuffle:
        combined_matrix = self.shuffle_columns(combined_matrix)

    return combined_matrix

In [25]:
a_col = 256
DCT1_wavelet_qt(16,a_col).shape

dct_matrix.shape (16, 128)
wavelet_matrix.shape (16, 8)


(16, 136)

In [21]:
256 // 2

128